In [1]:
print("Hello")

Hello


In [2]:
!qai-hub configure --api_token atbrq5wg15d5xbvszpmhw8k17kaldjq58p5luzou

qai-hub configuration saved to C:\Users\qc_de/.qai_hub/client.ini
==================== C:\Users\qc_de/.qai_hub/client.ini ====================
[api]
api_token = atbrq5wg15d5xbvszpmhw8k17kaldjq58p5luzou
api_url = https://app.aihub.qualcomm.com
web_url = https://app.aihub.qualcomm.com
verbose = True




2025-07-12 01:52:34.078 - INFO - Enabling verbose logging.
C:\Users\qc_de\anaconda3\envs\python-310\lib\site-packages\qai_hub\_cli.py:384: UserWarning: Overwriting configuration: C:\Users\qc_de/.qai_hub/client.ini (previous configuration saved to C:\Users\qc_de/.qai_hub/client.ini.bak)
  warnings.warn(


In [24]:
!qai-hub list-devices

+-------------------------------+--------------+----------+---------+---------------------------------------------------+---------------------------------------------------------+
|             Device            |      OS      |  Vendor  |   Type  |                      Chipset                      |                      CLI Invocation                     |
+-------------------------------+--------------+----------+---------+---------------------------------------------------+---------------------------------------------------------+
|    Google Pixel 3 (Family)    |  Android 10  |  Google  |  Phone  |          qualcomm-snapdragon-845, sdm845          |    --device "Google Pixel 3 (Family)" --device-os 10    |
|         Google Pixel 3        |  Android 10  |  Google  |  Phone  |          qualcomm-snapdragon-845, sdm845          |         --device "Google Pixel 3" --device-os 10        |
|        Google Pixel 3a        |  Android 10  |  Google  |  Phone  |          qualcomm-snapdragon-6

# Ideal Case

In [ ]:
import qai_hub as hub
import torch
from torchvision.models import mobilenet_v2
import requests
import numpy as np
from PIL import Image

# Using pre-trained MobileNet
torch_model = mobilenet_v2(pretrained=True)
torch_model.eval()

# Step 1: Trace model
input_shape = (1, 3, 224, 224)
example_input = torch.rand(input_shape)
traced_torch_model = torch.jit.trace(torch_model, example_input)

# Step 2: Compile model
compile_job = hub.submit_compile_job(
    model=traced_torch_model,
    device=hub.Device("Snapdragon X Elite CRD"),
    input_specs=dict(image=input_shape),
    options="--target_runtime onnx",
)

# Step 3: Profile on cloud-hosted device
target_model = compile_job.get_target_model()
profile_job = hub.submit_profile_job(
    model=target_model,
    device=hub.Device("Snapdragon X Elite CRD"),
)

# Step 4: Run inference on cloud-hosted device
sample_image_url = (
    "https://qaihub-public-assets.s3.us-west-2.amazonaws.com/apidoc/input_image1.jpg"
)
response = requests.get(sample_image_url, stream=True)
response.raw.decode_content = True
image = Image.open(response.raw).resize((224, 224))
input_array = np.expand_dims(
    np.transpose(np.array(image, dtype=np.float32) / 255.0, (2, 0, 1)), axis=0
)

# Run inference using the on-device model on the input image
inference_job = hub.submit_inference_job(
    model=target_model,
    device=hub.Device("Snapdragon X Elite CRD"),
    inputs=dict(image=[input_array]),
)
on_device_output = inference_job.download_output_data()

# Step 5: Post-processing the on-device output
output_name = list(on_device_output.keys())[0]
out = on_device_output[output_name][0]
on_device_probabilities = np.exp(out) / np.sum(np.exp(out), axis=1)

# Read the class labels for imagenet
sample_classes = "https://qaihub-public-assets.s3.us-west-2.amazonaws.com/apidoc/imagenet_classes.txt"
response = requests.get(sample_classes, stream=True)
response.raw.decode_content = True
categories = [str(s.strip()) for s in response.raw]

# Print top five predictions for the on-device model
print("Top-5 On-Device predictions:")
top5_classes = np.argsort(on_device_probabilities[0], axis=0)[-5:]
for c in reversed(top5_classes):
    print(f"{c} {categories[c]:20s} {on_device_probabilities[0][c]:>6.1%}")

# Step 6: Download model
target_model = compile_job.get_target_model()
target_model.download("mobilenet_v2.onnx")


c:\Users\qc_de\anaconda3\envs\python-310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\qc_de\anaconda3\envs\python-310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Uploading tmpag7uif2y.pt


100%|██████████| 13.9M/13.9M [00:02<00:00, 5.34MB/s]


Scheduled compile job (j56zzyz0g) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/j56zzyz0g/

Waiting for compile job (j56zzyz0g) completion. Type Ctrl+C to stop waiting at any time.
    ✅ SUCCESS                          
Scheduled profile job (jg9yylwl5) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jg9yylwl5/



Uploading dataset: 154kB [00:00, 384kB/s]                    <?, ?B/s]


Scheduled inference job (jp1ww4e2g) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jp1ww4e2g/

Waiting for inference job (jp1ww4e2g) completion. Type Ctrl+C to stop waiting at any time.


KeyboardInterrupt: 

# TryOut

In [9]:
import cv2
import torch
import numpy as np
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from PIL import Image
import torch.onnx

# 1. Load pretrained X3D-XS model
model = torch.hub.load("facebookresearch/pytorchvideo", "x3d_xs", pretrained=True)
model.eval()

# 2. Define transforms (Image → Tensor)
transform = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225])
])

# 3. Capture 16 frames from webcam
cap = cv2.VideoCapture(0)
frames = []

print("Capturing 16 frames from webcam...")
while len(frames) < 16:
    ret, frame = cap.read()
    if not ret:
        continue
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_img = Image.fromarray(rgb)
    tensor = transform(pil_img)
    frames.append(tensor)
cap.release()

# 4. Format: (T, C, H, W) → (1, C, T, H, W)
video_tensor = torch.stack(frames, dim=0).permute(1, 0, 2, 3).unsqueeze(0)

# 5. Export to ONNX
onnx_path = "x3d_xs_webcam.onnx"
print(f"Exporting model to {onnx_path}...")

torch.onnx.export(
    model,
    video_tensor,  # (1, 3, 16, 224, 224)
    "x3d_xs_webcam_static.onnx",
    input_names=["video"],
    output_names=["logits"],
    opset_version=13,
    dynamic_axes=None  # ✅ Remove dynamic_axes
)

print(f"✅ Model exported successfully: {onnx_path}")


Using cache found in C:\Users\qc_de/.cache\torch\hub\facebookresearch_pytorchvideo_main


Capturing 16 frames from webcam...
Exporting model to x3d_xs_webcam.onnx...
✅ Model exported successfully: x3d_xs_webcam.onnx


In [10]:
from qai_hub import submit_compile_job, submit_inference_job, Device

# Submit compile job
compile_job = submit_compile_job(
    model="x3d_xs_webcam_static.onnx",
    device=Device("Snapdragon X Elite CRD"),
    options="--target_runtime onnx"
)
target_model = compile_job.get_target_model()

# Run inference
inference_job = submit_inference_job(
    model=target_model,
    device=Device("Snapdragon X Elite CRD"),
    inputs={"video": [video_tensor.numpy()]},
)
result = inference_job.download_output_data()
output_logits = list(result.values())[0][0]
probs = torch.tensor(output_logits).softmax(dim=-1)

# Print top predictions
top5 = torch.topk(probs, 5)
print("Top-5 predicted class indices:")
for idx, prob in zip(top5.indices[0], top5.values[0]):
    print(f"Class {idx.item():3d} : {prob.item()*100:.1f}%")

Uploading x3d_xs_webcam_static.onnx


100%|██████████| 14.5M/14.5M [00:02<00:00, 6.44MB/s]


Scheduled compile job (jgzjj37op) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jgzjj37op/

Waiting for compile job (jgzjj37op) completion. Type Ctrl+C to stop waiting at any time.
    ✅ SUCCESS                          


Uploading dataset: 1.90MB [00:00, 4.10MB/s]                            5.03MB/s]


Scheduled inference job (jp1ww488g) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jp1ww488g/

Waiting for inference job (jp1ww488g) completion. Type Ctrl+C to stop waiting at any time.
    ✅ SUCCESS                          


tmpn3dkvmvm.h5: 100%|██████████| 13.9k/13.9k [00:00<00:00, 14.3MB/s]

Top-5 predicted class indices:
Class 196 : 68.2%
Class  36 : 3.0%
Class  79 : 2.6%
Class 392 : 2.5%
Class 316 : 1.4%


In [18]:
# Step 1: Download the label file
url = "https://dl.fbaipublicfiles.com/pyslowfast/dataset/class_names/kinetics_classnames.json"
response = requests.get(url)
response.raise_for_status()

# Step 2: Load and invert the mapping
class_map = json.loads(response.text)
index_to_label = {v: k for k, v in class_map.items()}

# Step 3: Create a list where index matches label position
labels = [index_to_label[i] for i in range(len(index_to_label))]

## Webcam Feed

In [9]:
import cv2
import torch
import numpy as np
import onnxruntime as ort
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from PIL import Image
import requests

# Load class labels (Kinetics-400)
label_url = "https://raw.githubusercontent.com/deepmind/kinetics-i3d/master/data/label_map.txt"
labels = requests.get(label_url).text.strip().split("\n")

# Preprocessing pipeline
transform = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize([0.45, 0.45, 0.45], [0.225, 0.225, 0.225])
])

# Load ONNX model
session = ort.InferenceSession("x3d_xs_webcam_static.onnx")
input_name = session.get_inputs()[0].name

# Webcam feed
cap = cv2.VideoCapture(0)
frames = []

print("🎥 Starting real-time activity recognition...")

while True:
    ret, frame = cap.read()
    if not ret:
        continue

    # Convert and preprocess
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_img = Image.fromarray(rgb)
    tensor_img = transform(pil_img)
    frames.append(tensor_img)

    if len(frames) == 16:
        # Prepare input: (T, C, H, W) → (1, C, T, H, W)
        video_tensor = torch.stack(frames).permute(1, 0, 2, 3).unsqueeze(0).numpy()
        frames = []  # Reset buffer

        # Inference
        outputs = session.run(None, {input_name: video_tensor})
        probs = torch.tensor(outputs[0]).softmax(dim=-1)[0]
        top5 = torch.topk(probs, 5)

        # Draw predictions on frame
        for i, (idx, prob) in enumerate(zip(top5.indices, top5.values)):
            label = labels[idx.item()]
            text = f"{label}: {prob.item()*100:.1f}%"
            cv2.putText(frame, text, (10, 30 + i*25),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            print(label, prob.item() * 100)

    # Show frame
    cv2.imshow("Real-Time Activity Recognition", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


🎥 Starting real-time activity recognition...
beatboxing 73.50980639457703
playing harmonica 7.458873838186264
shaking head 5.060039833188057
cracking neck 4.629659280180931
smoking 0.9155870415270329
beatboxing 64.70531225204468
counting money 7.534674555063248
singing 4.750609770417213
playing harmonica 3.7736229598522186
fixing hair 2.648218162357807
beatboxing 26.610475778579712
singing 11.11048236489296
fixing hair 8.783400058746338
answering questions 6.3661277294158936
playing ukulele 5.524526908993721
beatboxing 43.91208291053772
playing harmonica 17.475034296512604
whistling 2.5240490213036537
playing recorder 2.444513700902462
answering questions 2.2485192865133286
tapping guitar 8.45736563205719
playing ukulele 5.776163190603256
playing guitar 3.9676733314990997
playing bass guitar 2.345132641494274
playing recorder 2.1889496594667435
beatboxing 24.648168683052063
playing harmonica 16.092631220817566
finger snapping 8.953815698623657
playing recorder 8.083099126815796
whistli

## Webcam: Average 5 Frames

In [3]:
import cv2
import torch
import numpy as np
import onnxruntime as ort
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from PIL import Image
import requests
import time

last_prediction = []       # stores list of (label, prob)
last_prediction_time = 0   # timestamp of last update
display_duration = 2       # seconds to keep text on screen


# Load class labels (Kinetics-400)
label_url = "https://raw.githubusercontent.com/deepmind/kinetics-i3d/master/data/label_map.txt"
labels = requests.get(label_url).text.strip().split("\n")

# Preprocessing pipeline
transform = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize([0.45, 0.45, 0.45], [0.225, 0.225, 0.225])
])

# Load ONNX model
session = ort.InferenceSession("x3d_xs_webcam_static.onnx")
input_name = session.get_inputs()[0].name

# Webcam feed
cap = cv2.VideoCapture(0)
frames = []

print("🎥 Starting real-time activity recognition...")

from collections import deque

rolling_window = deque(maxlen=5)  # last 5 inference results

while True:
    ret, frame = cap.read()
    if not ret:
        continue

    # Convert and preprocess
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_img = Image.fromarray(rgb)
    tensor_img = transform(pil_img)
    frames.append(tensor_img)

    if len(frames) == 16:
        # Prepare input: (T, C, H, W) → (1, C, T, H, W)
        video_tensor = torch.stack(frames).permute(1, 0, 2, 3).unsqueeze(0).numpy()
        frames = []  # Reset buffer

        # Run ONNX inference
        outputs = session.run(None, {input_name: video_tensor})
        probs = torch.tensor(outputs[0]).softmax(dim=-1)[0]

        # Add to rolling window
        rolling_window.append(probs)
        if rolling_window:
            avg_probs = torch.stack(list(rolling_window)).mean(dim=0)
            top5 = torch.topk(avg_probs, 1)
            last_prediction = [(labels[idx.item()], prob.item()) for idx, prob in zip(top5.indices, top5.values)]
            last_prediction_time = time.time()

        if time.time() - last_prediction_time < display_duration:
            for i, (label, prob) in enumerate(last_prediction):
                if prob > 0.3:  # Optional: skip low confidence
                    text = f"{label}: {prob*100:.1f}%"
                    cv2.putText(frame, text, (10, 30 + i*25),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                    print(label, prob * 100)
        
        # # Draw predictions on frame
        # for i, (idx, prob) in enumerate(zip(top5.indices, top5.values)):
        #     label = labels[idx.item()]
        #     text = f"{label}: {prob.item()*100:.1f}%"
        #     cv2.putText(frame, text, (10, 30 + i*25),
        #                 cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        #     print(label, prob.item() * 100)

    # Show frame
    cv2.imshow("Real-Time Activity Recognition", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


🎥 Starting real-time activity recognition...
massaging person's head 31.671777367591858
massaging person's head 35.23405194282532
massaging person's head 30.698195099830627
massaging person's head 35.83304286003113
massaging person's head 40.16903042793274
massaging person's head 39.080601930618286
massaging person's head 35.29893159866333
finger snapping 30.935224890708923
finger snapping 40.45135974884033
finger snapping 45.773300528526306
finger snapping 38.88634741306305
finger snapping 37.03685700893402
finger snapping 40.65384864807129
finger snapping 34.5172256231308
finger snapping 33.37318301200867
finger snapping 30.45106530189514
massaging person's head 33.14814567565918
massaging person's head 33.16800892353058
finger snapping 33.7223619222641
finger snapping 34.03366208076477


## Video

In [6]:
import cv2
import torch
import numpy as np
import onnxruntime as ort
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from PIL import Image
import requests
import time
from collections import deque

# Load labels
label_url = "https://dl.fbaipublicfiles.com/pyslowfast/dataset/class_names/kinetics_classnames.json"
class_map = requests.get(label_url).json()
labels = [None] * len(class_map)
for name, idx in class_map.items():
    labels[int(idx)] = name

# Transforms
transform = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize([0.45]*3, [0.225]*3)
])

# Load ONNX model
session = ort.InferenceSession("x3d_xs_webcam_static.onnx")
input_name = session.get_inputs()[0].name

# Rolling average
rolling_window = deque(maxlen=5)
display_duration = 2
last_pred_time = 0
last_prediction = []

# Load video file with humans
cap = cv2.VideoCapture("videos/driver-action-recognition.mp4")

frames = []
print("Processing video with human actions...")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess frame
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = transform(Image.fromarray(rgb))
    frames.append(img)

    # Inference every 16 frames
    if len(frames) == 16:
        video_tensor = torch.stack(frames).permute(1, 0, 2, 3).unsqueeze(0).numpy()
        outputs = session.run(None, {input_name: video_tensor})
        probs = torch.tensor(outputs[0]).softmax(dim=-1)[0]
        rolling_window.append(probs)
        avg_probs = torch.stack(list(rolling_window)).mean(dim=0)
        top5 = torch.topk(avg_probs, 5)
        last_prediction = [(labels[idx.item()], prob.item()) for idx, prob in zip(top5.indices, top5.values)]
        last_pred_time = time.time()
        frames.clear()

    # Draw predictions if recent
    if time.time() - last_pred_time < display_duration:
        for i, (label, prob) in enumerate(last_prediction):
            if prob > 0.3:
                text = f"{label}: {prob*100:.1f}%"
                cv2.putText(frame, text, (10, 30 + i*25),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

    cv2.imshow("Activity Recognition – Video File", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


Processing video with human actions...


# VideoMAE

In [ ]:
import cv2
import torch
import numpy as np
import onnxruntime as ort
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from PIL import Image
from collections import deque
import time
import requests
import json

# video_mae model Source:
# https://aihub.qualcomm.com/models/video_mae?searchTerm=action&domain=Computer+Vision&useCase=Video+Classification

# Load VideoMAE ONNX model
session = ort.InferenceSession("models/video_mae.onnx")
input_name = session.get_inputs()[0].name

# Load class labels (Kinetics-400)
label_url = "https://dl.fbaipublicfiles.com/pyslowfast/dataset/class_names/kinetics_classnames.json"
labels_map = requests.get(label_url).json()
index_to_label = {v: k for k, v in labels_map.items()}
labels = [index_to_label[i] for i in range(len(index_to_label))]

# Define transform
transform = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize([0.5]*3, [0.5]*3)
])

# Rolling average buffer
rolling_window = deque(maxlen=5)
display_duration = 2
last_pred_time = 0
last_prediction = []

# Open webcam
cap = cv2.VideoCapture(0)
frames = []

print("🎥 Starting real-time VideoMAE inference...")

# Store last 50 predictions
recent_predictions = deque(maxlen=50)

while True:
    ret, frame = cap.read()
    if not ret:
        continue

    # Preprocess frame
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_img = Image.fromarray(rgb)
    tensor_img = transform(pil_img)
    frames.append(tensor_img)

    # Once we have 16 frames, run inference
    if len(frames) == 16:
        clip = torch.stack(frames).permute(1, 0, 2, 3).unsqueeze(0).numpy()  # (1, 3, 16, 224, 224)
        frames.clear()

        # Run ONNX inference
        outputs = session.run(None, {input_name: clip})
        probs = torch.tensor(outputs[0]).softmax(dim=-1)[0]
        rolling_window.append(probs)

        # Average predictions over buffer
        avg_probs = torch.stack(list(rolling_window)).mean(dim=0)
        top5 = torch.topk(avg_probs, 5)
        # last_prediction = [(labels[idx.item()], prob.item()) for idx, prob in zip(top5.indices, top5.values)]
        # last_pred_time = time.time()
        
        top5_results = [(labels[idx.item()], prob.item()) for idx, prob in zip(top5.indices, top5.values)]
        last_prediction = top5_results
        last_pred_time = time.time()

        # Store only the top-1 label for analysis
        top1_label = top5_results[0][0]
        recent_predictions.append(top1_label)

    # Draw predictions if within display window
    if time.time() - last_pred_time < display_duration:
        for i, (label, prob) in enumerate(last_prediction):
            if prob > 0.3:
                text = f"{label}: {prob*100:.1f}%"
                cv2.putText(frame, text, (10, 30 + i*25),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    cv2.imshow("VideoMAE - Webcam Activity Recognition", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

with open("activity_predictions.txt", "w") as f:
    for label in recent_predictions:
        f.write(label + "\n")
        
cap.release()
cv2.destroyAllWindows()


🎥 Starting real-time VideoMAE inference...


# Mistral

In [4]:
import pkg_resources
print(pkg_resources.get_distribution("qai-hub-models").version)


0.31.0


In [11]:
# from qai_hub_models.models.mistral_7b_instruct_v0_3 import Model
# from qai_hub_models.models.mistral_7b_instruct_v0_3 import Mistral_7B_Instruct_V0_3
import os, sys

pkg_path = os.path.join(sys.prefix, "Lib", "site-packages", "qai_hub_models", "models")
print(os.listdir(pkg_path))

['allam_7b', 'aotgan', 'baichuan2_7b', 'beit', 'bgnet', 'bisenet', 'common.py', 'conditional_detr_resnet50', 'controlnet', 'convnext_base', 'convnext_tiny', 'ddrnet23_slim', 'deepbox', 'deeplabv3_plus_mobilenet', 'deeplabv3_resnet50', 'densenet121', 'depth_anything', 'depth_anything_v2', 'detr_resnet101', 'detr_resnet101_dc5', 'detr_resnet50', 'detr_resnet50_dc5', 'dla102x', 'easyocr', 'efficientnet_b0', 'efficientnet_b4', 'efficientnet_v2_s', 'efficientvit_b2_cls', 'efficientvit_l2_cls', 'efficientvit_l2_seg', 'esrgan', 'facemap_3dmm', 'face_attrib_net', 'face_det_lite', 'fastsam_s', 'fastsam_x', 'fcn_resnet50', 'ffnet_122ns_lowres', 'ffnet_40s', 'ffnet_54s', 'ffnet_78s', 'ffnet_78s_lowres', 'fomm', 'foot_track_net', 'gear_guard_net', 'googlenet', 'hrnet_pose', 'hrnet_w48_ocr', 'huggingface_wavlm_base_plus', 'ibm_granite_v3_1_8b_instruct', 'inception_v3', 'indus_1b', 'jais_6p7b_chat', 'lama_dilated', 'levit', 'litehrnet', 'llama_v2_7b_chat', 'llama_v3_1_8b_instruct', 'llama_v3_1_sea_l

In [12]:
import importlib

# Load the model module dynamically
mistral_mod = importlib.import_module("qai_hub_models.models.mistral_7b_instruct_v0_3")

# Get the class name manually
print([x for x in dir(mistral_mod) if not x.startswith("_")])


['MODEL_ID', 'Model', 'model']


In [17]:
from qai_hub_models.models.mistral_7b_instruct_v0_3 import Model
from qai_hub import Device
# Instantiate the model (QNN or default variant)
model = Model.from_pretrained()

# Run inference
response = model("Assess the threat level of the following actions: punching, running, hugging.")
print(response)


AttributeError: type object 'Mistral_7B_Instruct_v0_3' has no attribute 'from_pretrained'

In [21]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from optimum.exporters.onnx import main_export
from optimum.exporters.onnx.config import OnnxConfigWithPast
# from optimum.exporters.onnx.model_export import export
from optimum.exporters.onnx.model_configs import TextDecoderOnnxConfig
from pathlib import Path

model_id = "mistralai/Mistral-7B-Instruct-v0.2"
output_dir = Path("mistral_onnx")

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Define ONNX export config
onnx_config = TextDecoderOnnxConfig(model.config)

# Export model
export(
    model=model,
    config=onnx_config,
    opset=13,
    output=output_dir,
    tokenizer=tokenizer,
    device="cpu"
)

print(f"✅ Export completed: ONNX model saved to {output_dir.resolve()}")


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2.
401 Client Error. (Request ID: Root=1-6872bc25-31fb174005927b884d3ad788;e2a22024-b1fc-462a-b253-67455695f74c)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/config.json.
Access to model mistralai/Mistral-7B-Instruct-v0.2 is restricted. You must have access to it and be authenticated to access it. Please log in.

# Sandbox

In [2]:
import cv2
import numpy as np
import onnxruntime as ort

from collections import deque
import onnx
import urllib.request

In [3]:
session = ort.InferenceSession("models/yolov8n.onnx")

In [4]:
# Inspect ONNX model input details
for input in session.get_inputs():
    print("Name:", input.name)
    print("Shape:", input.shape)
    print("Type:", input.type)

Name: images
Shape: [1, 3, 640, 640]
Type: tensor(float)


In [7]:
from qai_hub_models.models.ultralytics_yolov8 import UltralyticsYOLOv8
from qai_hub_models.models.mediapipe_blazepose import MediapipeBlazePose

# Instantiate models
yolo = UltralyticsYOLOv8(model_variant="nano-int8")
pose_model = MediapipeBlazePose(model_variant="lightning-int8")

# Download and prepare for inference
yolo.prepare()
pose_model.prepare()

ModuleNotFoundError: No module named 'qai_hub_models'